In [ ]:
loss

In [4]:
from keras.applications import inception_v3
from keras import backend as K
import scipy
import numpy as np
from keras.preprocessing import image

K.set_learning_phase(0)
model=inception_v3.InceptionV3(weights="imagenet",include_top=False)

layer_contributions={"mixed2":0.2,"mixed3":1.,"mixed4":2.,"mixed5":1.0,}
layer_dict = dict([(layer.name,layer) for layer in model.layers])
loss=K.variable(0.)

for layer_name in layer_contributions:
    coeff=layer_contributions[layer_name]
    activation=layer_dict[layer_name].output
    scaling=K.prod(K.cast(K.shape(activation),"float32"))
    loss=loss+coeff * K.sum(K.square(activation[:, 2: -2, 2: -2,:]))/scaling

dream = model.input
grads=K.gradients(loss,dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)),1e-7)
outputs=[loss,grads]
fetch_loss_and_grads=K.function([dream],outputs)

def eval_loss_and_grads(x):
    outs=fetch_loss_and_grads([x])
    loss_values=outs[0]
    grad_values=outs[1]
    return loss_values, grad_values

def gradient_ascent(x,iterations,step,max_loss=None):
    for i in range(iterations):
        loss_values,grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_values > max_loss:
            break
        print("...Loss values at",i,":",loss_values)
        x+=step*grad_values
    return x

def resize_img(img,size):
    img=np.copy(img)
    factors=(1,float(size[0])/img.shape[1],float(size[1])/img.shape[2],1)
    return scipy.ndimage.zoom(img,factors,order=1)

def save_img(img,fname):
    pil_img=deprocess_image(np.copy(img))
    scipy.misc.imsave(fname,pil_img)
    
def preprocess_image(image_path):
    img=image.load_img(image_path)
    img=image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x=x.reshape((3,x.shape[2],x.shape[3]))
        x=transpose((1,2,0))
    else:
        x=x.reshape((x.shape[1], x.shape[2],3))
    x/=2.
    x+=0.5
    x*255.
    x=np.clip(x,0,255).astype("uint8")
    return x

step = 0.01
num_octave=3
octave_scale=1.4
iterations=20
max_loss=10.
base_image_path="images.jpg"

img=preprocess_image(base_image_path)
original_shape=img.shape[1:3]
successive_shapes=[original_shape]
for i in range(1,num_octave):
    shape=tuple([int(dim/(octave_scale**i))
        for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes=successive_shapes[::-1]
original_img=np.copy(img)
shrunk_original_img=resize_img(img,successive_shapes[0])
for shape in successive_shapes:
    print("Processing image shape",shape)
    img= resize_img(img,shape)
    img=gradient_ascent(img,iterations=iterations,step=step,max_loss=max_loss)
    upscaled_shrunk_original_img=resize_img(shrunk_original_img, shape)
    same_size_original=resize_img(original_img, shape)
    lost_detail= same_size_original - upscaled_shrunk_original_img
    img+= lost_detail
    shrunk_original_img=resize_img(original_img, shape)
    save_img(img,fname="dream_at_scale"+str(shape)+".png")
save_img(img,fname="final_dream.png")


Processing image shape (153, 153)
...Loss values at 0 : 0.49244735
...Loss values at 1 : 0.6594336
...Loss values at 2 : 0.90809184
...Loss values at 3 : 1.1650867
...Loss values at 4 : 1.3647465
...Loss values at 5 : 1.5232582
...Loss values at 6 : 1.7008245
...Loss values at 7 : 1.8776016
...Loss values at 8 : 2.0577273
...Loss values at 9 : 2.269582
...Loss values at 10 : 2.391016
...Loss values at 11 : 2.5445552
...Loss values at 12 : 2.6836834
...Loss values at 13 : 2.8376713
...Loss values at 14 : 3.0453196
...Loss values at 15 : 3.1784506
...Loss values at 16 : 3.3831663
...Loss values at 17 : 3.620345
...Loss values at 18 : 3.8187227
...Loss values at 19 : 3.9317548


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Processing image shape (214, 214)
...Loss values at 0 : 1.1884912
...Loss values at 1 : 1.8369424
...Loss values at 2 : 2.3583403
...Loss values at 3 : 2.8446014
...Loss values at 4 : 3.237917
...Loss values at 5 : 3.5768852
...Loss values at 6 : 3.9069967
...Loss values at 7 : 4.1850295
...Loss values at 8 : 4.508936
...Loss values at 9 : 4.7608314
...Loss values at 10 : 5.084732
...Loss values at 11 : 5.299462
...Loss values at 12 : 5.5436354
...Loss values at 13 : 5.7561884
...Loss values at 14 : 6.037732
...Loss values at 15 : 6.3290105
...Loss values at 16 : 6.554292
...Loss values at 17 : 6.795218
...Loss values at 18 : 7.0172453
...Loss values at 19 : 7.2561064
Processing image shape (300, 300)
...Loss values at 0 : 1.4981992
...Loss values at 1 : 2.2032983
...Loss values at 2 : 2.8403816
...Loss values at 3 : 3.3540728
...Loss values at 4 : 3.8288012
...Loss values at 5 : 4.2640834
...Loss values at 6 : 4.6880603
...Loss values at 7 : 5.1324663
...Loss values at 8 : 5.634111
..